In [1]:
import pandas as pd

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\data_performance\2025_12_12__17_50_08_659.csv"

# Skip the first 9 rows (metadata header), allow variable number of fields, handle empty cells
df = pd.read_csv(
    file_path,
    skiprows=9,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

df

C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 14: Expected 8 fields in line 14, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 15: Expected 8 fields in line 15, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 16: Expected 8 fields in line 16, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 24: Expected 8 fields in line 24, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 26: Expected 8 fields in line 26, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6: ParserWarning: Skipping line 31: Expected 8 fields in line 31, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_5956\3597505872.py:6

,Num_line,S,MS,Cat,Num_cat,State,Display,null
0,1,0,0,Entry,0.0,Ready,NaN,NaN
1,2,0,5,Exit,0.0,Ready,Time,NaN
2,3,0,5,Entry,1.0,MagEntry,NaN,NaN
3,7,3,575,Input,1.0,On1A1,NaN,NaN
4,8,3,575,Exit,1.0,MagEntry,1,NaN
...,...,...,...,...,...,...,...,...
5001,7452,2398,760,Exit,1.0,MagEntry,1,NaN
5002,7453,2398,760,Entry,2.0,ITI2sec,NaN,NaN
5003,7454,2399,425,Input,33.0,Off1A1,NaN,NaN
5004,7455,2400,0,Exit,2.0,ITI2sec,Time,NaN


In [2]:
# Identify trials: each trial starts with "MagEntry" where Cat is "Entry" and ends with the next "MagEntry" with Cat "Entry"
# Find all indices where "MagEntry" appears in State column AND "Entry" in Cat column
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()

# Create a list to store trial information
trials = []

# For each pair of consecutive MagEntry Entry events
for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Extract the trial data (from start to end, inclusive of end)
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Get all states/categories in this trial
    trial_states = trial_data['State'].tolist()
    
    # Store trial information using actual dataframe indices
    trials.append({
        'trial_number': i + 1,
        'start_line': trial_start_idx,
        'end_line': trial_end_idx,
        'start_idx': trial_start_idx,
        'end_idx': trial_end_idx,
        'states': trial_states,
        'num_events': len(trial_data)
    })

# Display trials summary
for trial in trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_line']} to {trial['end_line']} ({trial['num_events']} events)")
    print(f"  States: {' -> '.join(trial['states'])}")
    print()

print(f"Total trials identified: {len(trials)}")

Trial 1: Rows 2 to 18 (17 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole1 -> hole1 -> Holdhole1 -> Holdhole1 -> Omission -> Omission -> TimeOut -> TimeOut -> MagEntry

Trial 2: Rows 18 to 44 (27 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole3 -> Off1A1 -> hole3 -> Holdhole3 -> Holdhole3 -> Omission -> Omission -> TimeOut -> On1A4 -> Off1A4 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> TimeOut -> MagEntry

Trial 3: Rows 44 to 68 (25 events)
  States: MagEntry -> On1A3 -> On1A4 -> Off1A3 -> Off1A4 -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole4 -> Off1A1 -> hole4 -> Holdhole4 -> On1A2 -> On1A3 -> Holdhole4 -> CorrectResp -> CorrectResp -> MagEntry

Trial 4: Rows 68 to 96 (29 events)
  States: MagEntry -> Off1A2 -> Off1A3 -> On1A1 -> MagEntry -> ITI2sec -> Off1A

In [4]:
# Load df_659_12_12.csv and create detailed trials summary with timestamps
import pandas as pd
import json

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\df_659_12_12.csv"

# Load the data - NO skiprows since first row is header only
df = pd.read_csv(
    file_path,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

# Display the dataframe with actual row numbers to verify
print("Data loaded. Showing first 22 rows:")
for idx in range(min(22, len(df))):
    actual_row = idx + 2  # Convert pandas index to actual row number (row 1 = header)
    print(f"Row {actual_row}: State={df.loc[idx, 'State']}, Cat={df.loc[idx, 'Cat']}")
print(f"\nDataframe has {len(df)} rows of data\n")

# Identify trials: MagEntry with Entry
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()
print(f"Found MagEntry+Entry at pandas indices: {magentry_indices}")
print(f"Found MagEntry+Entry at actual CSV rows: {[idx + 2 for idx in magentry_indices]}\n")

# Create detailed trials list with state information and timestamps
detailed_trials = []

for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Convert to actual row numbers (add 2: 1 for header row + 1 for 1-based indexing)
    actual_start_row = trial_start_idx + 2
    actual_end_row = trial_end_idx + 2
    
    # Extract trial data
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Create list of states with timestamps
    states_with_time = []
    for idx, row in trial_data.iterrows():
        actual_row_num = idx + 2
        states_with_time.append({
            'actual_row': actual_row_num,
            'state': row['State'],
            'cat': row['Cat'],
            'seconds': row['S'],
            'milliseconds': row['MS'],
            'display': row['Display']
        })
    
    # Store trial information with ACTUAL row numbers
    detailed_trials.append({
        'trial_number': i + 1,
        'start_row': actual_start_row,
        'end_row': actual_end_row,
        'num_events': len(trial_data),
        'states_sequence': ' -> '.join([s['state'] for s in states_with_time]),
        'states_detailed': json.dumps(states_with_time)
    })

# Create DataFrame and save to CSV
trials_df = pd.DataFrame(detailed_trials)

output_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\trials_summary.csv"
trials_df.to_csv(output_path, index=False)

print(f"Detailed trials summary saved to: {output_path}")
print(f"Total trials identified: {len(detailed_trials)}\n")

# Display all trials with ACTUAL row numbers
for trial in detailed_trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_row']} to {trial['end_row']} ({trial['num_events']} events)")
    print(f"  States: {trial['states_sequence']}")
    print()

trials_df

Data loaded. Showing first 22 rows:
Row 2: State=Ready, Cat=Entry
Row 3: State=Ready, Cat=Exit
Row 4: State=MagEntry, Cat=Entry
Row 5: State=On1A1, Cat=Input
Row 6: State=MagEntry, Cat=Exit
Row 7: State=ITI2sec, Cat=Entry
Row 8: State=Off1A1, Cat=Input
Row 9: State=ITI2sec, Cat=Exit
Row 10: State=RandomHole, Cat=Entry
Row 11: State=RandomHole, Cat=Exit
Row 12: State=hole1, Cat=Entry
Row 13: State=hole1, Cat=Exit
Row 14: State=Holdhole1, Cat=Entry
Row 15: State=Holdhole1, Cat=Exit
Row 16: State=Omission, Cat=Entry
Row 17: State=Omission, Cat=Exit
Row 18: State=TimeOut, Cat=Entry
Row 19: State=TimeOut, Cat=Exit
Row 20: State=MagEntry, Cat=Entry
Row 21: State=On1A1, Cat=Input
Row 22: State=MagEntry, Cat=Exit
Row 23: State=ITI2sec, Cat=Entry

Dataframe has 5006 rows of data

Found MagEntry+Entry at pandas indices: [2, 18, 44, 68, 96, 122, 136, 152, 174, 192, 218, 232, 250, 286, 301, 319, 340, 358, 384, 397, 420, 434, 452, 468, 484, 499, 513, 527, 541, 562, 586, 605, 627, 651, 667, 686, 705

,trial_number,start_row,end_row,num_events,states_sequence,states_detailed
0,1,4,20,17,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 4, ""state"": ""MagEntry"", ""cat"":..."
1,2,20,46,27,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 20, ""state"": ""MagEntry"", ""cat""..."
2,3,46,70,25,MagEntry -> On1A3 -> On1A4 -> Off1A3 -> Off1A4...,"[{""actual_row"": 46, ""state"": ""MagEntry"", ""cat""..."
3,4,70,98,29,MagEntry -> Off1A2 -> Off1A3 -> On1A1 -> MagEn...,"[{""actual_row"": 70, ""state"": ""MagEntry"", ""cat""..."
4,5,98,124,27,MagEntry -> On1A4 -> Off1A4 -> On1A4 -> Off1A4...,"[{""actual_row"": 98, ""state"": ""MagEntry"", ""cat""..."
...,...,...,...,...,...,...
287,288,4921,4934,14,MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Of...,"[{""actual_row"": 4921, ""state"": ""MagEntry"", ""ca..."
288,289,4934,4955,22,MagEntry -> On1A4 -> Off1A3 -> On1A3 -> Off1A4...,"[{""actual_row"": 4934, ""state"": ""MagEntry"", ""ca..."
289,290,4955,4970,16,MagEntry -> Off1A3 -> Off1A4 -> On1A1 -> MagEn...,"[{""actual_row"": 4955, ""state"": ""MagEntry"", ""ca..."
290,291,4970,4984,15,MagEntry -> Off1A3 -> On1A1 -> MagEntry -> ITI...,"[{""actual_row"": 4970, ""state"": ""MagEntry"", ""ca..."


In [4]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    # start_row and end_row are 1-indexed actual CSV rows (with row 1 being header)
    # So pandas index = actual_row - 2
    # We exclude end_row since that's the start of the next trial
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find MagEntry timestamp (Entry state) - should only be one now (the start)
    mag_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'MagEntry')]
    if not mag_entry.empty:
        mag_s = mag_entry.iloc[0]['S']
        mag_ms = mag_entry.iloc[0]['MS']
        trials_summary.at[idx, 'MagEntry'] = format_time(mag_s, mag_ms)
    
    # Find On1A1 timestamp (Input state)
    on1a1 = trial_data[(trial_data['Cat'] == 'Input') & (trial_data['State'] == 'On1A1')]
    if not on1a1.empty:
        on1a1_s = on1a1.iloc[0]['S']
        on1a1_ms = on1a1.iloc[0]['MS']
        trials_summary.at[idx, 'On1A1'] = format_time(on1a1_s, on1a1_ms)
    
    print(f"Processed trial {trial_num}: MagEntry={trials_summary.at[idx, 'MagEntry']}, On1A1={trials_summary.at[idx, 'On1A1']}")

# Save the updated trials_summary
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Updated trials_summary.csv")


Processed trial 1: MagEntry=0.5, On1A1=3.575
Processed trial 2: MagEntry=13.615, On1A1=63.715
Processed trial 3: MagEntry=73.250, On1A1=81.780
Processed trial 4: MagEntry=85.405, On1A1=86.305
Processed trial 5: MagEntry=96.345, On1A1=131.210
Processed trial 6: MagEntry=138.100, On1A1=140.300
Processed trial 7: MagEntry=146.940, On1A1=148.445
Processed trial 8: MagEntry=158.485, On1A1=177.495
Processed trial 9: MagEntry=184.395, On1A1=187.620
Processed trial 10: MagEntry=194.715, On1A1=198.595
Processed trial 11: MagEntry=205.540, On1A1=222.30
Processed trial 12: MagEntry=224.515, On1A1=225.485
Processed trial 13: MagEntry=232.570, On1A1=330.175
Processed trial 14: MagEntry=333.545, On1A1=334.450
Processed trial 15: MagEntry=336.860, On1A1=338.515
Processed trial 16: MagEntry=341.780, On1A1=342.970
Processed trial 17: MagEntry=349.740, On1A1=352.680
Processed trial 18: MagEntry=360.185, On1A1=361.410
Processed trial 19: MagEntry=370.440, On1A1=371.385
Processed trial 20: MagEntry=374.19

C:\Users\charl\AppData\Local\Temp\ipykernel_23768\2298634137.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  trials_summary.at[idx, 'MagEntry'] = format_time(mag_s, mag_ms)
C:\Users\charl\AppData\Local\Temp\ipykernel_23768\2298634137.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.575' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  trials_summary.at[idx, 'On1A1'] = format_time(on1a1_s, on1a1_ms)


In [6]:
import pandas as pd

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        milliseconds = float(parts[1])
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int((seconds - s) * 1000)
    return f"{s}.{ms}"

# Calculate Delta for each row
delta_values = []
for idx, row in trials_summary.iterrows():
    mag_entry_time = time_to_seconds(row['MagEntry'])
    on1a1_time = time_to_seconds(row['On1A1'])
    
    if mag_entry_time is not None and on1a1_time is not None:
        delta_seconds = on1a1_time - mag_entry_time
        delta_formatted = seconds_to_time(delta_seconds)
        delta_values.append(delta_formatted)
        print(f"Trial {row['trial_number']}: On1A1({on1a1_time:.3f}s) - MagEntry({mag_entry_time:.3f}s) = Delta({delta_formatted})")
    else:
        delta_values.append(None)
        print(f"Trial {row['trial_number']}: Missing timestamp data")

# Insert Delta column right after On1A1
col_position = trials_summary.columns.get_loc('On1A1') + 1
trials_summary.insert(col_position, 'Delta', delta_values)

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added Delta column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'Delta']].head(10)

Trial 1: On1A1(3.575s) - MagEntry(0.005s) = Delta(3.570)
Trial 2: On1A1(63.715s) - MagEntry(13.615s) = Delta(50.100)
Trial 3: On1A1(81.078s) - MagEntry(73.025s) = Delta(8.52)
Trial 4: On1A1(86.305s) - MagEntry(85.405s) = Delta(0.900)
Trial 5: On1A1(131.021s) - MagEntry(96.345s) = Delta(34.675)
Trial 6: On1A1(140.003s) - MagEntry(138.001s) = Delta(2.1)
Trial 7: On1A1(148.445s) - MagEntry(146.094s) = Delta(2.350)
Trial 8: On1A1(177.495s) - MagEntry(158.485s) = Delta(19.9)
Trial 9: On1A1(187.062s) - MagEntry(184.395s) = Delta(2.667)
Trial 10: On1A1(198.595s) - MagEntry(194.715s) = Delta(3.879)
Trial 11: On1A1(222.003s) - MagEntry(205.054s) = Delta(16.948)
Trial 12: On1A1(225.485s) - MagEntry(224.515s) = Delta(0.970)
Trial 13: On1A1(330.175s) - MagEntry(232.057s) = Delta(98.118)
Trial 14: On1A1(334.045s) - MagEntry(333.545s) = Delta(0.500)
Trial 15: On1A1(338.515s) - MagEntry(336.086s) = Delta(2.428)
Trial 16: On1A1(342.097s) - MagEntry(341.078s) = Delta(1.19)
Trial 17: On1A1(352.068s) - M

,trial_number,MagEntry,On1A1,Delta
0,1,0.500,3.575,3.570
1,2,13.615,63.715,50.100
2,3,73.250,81.780,8.52
3,4,85.405,86.305,0.900
4,5,96.345,131.210,34.675
5,6,138.100,140.300,2.1
6,7,146.940,148.445,2.350
7,8,158.485,177.495,19.9
8,9,184.395,187.620,2.667
9,10,194.715,198.595,3.879


In [8]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial to find ITI_start timestamp
iti_start_values = []
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find ITI2sec timestamp (Entry state)
    iti_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'ITI2sec')]
    if not iti_entry.empty:
        iti_s = iti_entry.iloc[0]['S']
        iti_ms = iti_entry.iloc[0]['MS']
        iti_timestamp = format_time(iti_s, iti_ms)
        iti_start_values.append(iti_timestamp)
        print(f"Trial {trial_num}: ITI_start = {iti_timestamp}")
    else:
        iti_start_values.append(None)
        print(f"Trial {trial_num}: No ITI2sec Entry found")

# Add ITI_start column to trials_summary
trials_summary['ITI_start'] = iti_start_values

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added ITI_start column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'Delta', 'ITI_start']].head(10)

Trial 1: ITI_start = 3.575
Trial 2: ITI_start = 63.715
Trial 3: ITI_start = 81.780
Trial 4: ITI_start = 86.305
Trial 5: ITI_start = 131.210
Trial 6: ITI_start = 140.300
Trial 7: ITI_start = 148.445
Trial 8: ITI_start = 177.495
Trial 9: ITI_start = 187.620
Trial 10: ITI_start = 198.595
Trial 11: ITI_start = 222.30
Trial 12: ITI_start = 225.485
Trial 13: ITI_start = 330.175
Trial 14: ITI_start = 334.450
Trial 15: ITI_start = 338.515
Trial 16: ITI_start = 342.970
Trial 17: ITI_start = 352.680
Trial 18: ITI_start = 361.410
Trial 19: ITI_start = 371.385
Trial 20: ITI_start = 379.855
Trial 21: ITI_start = 390.735
Trial 22: ITI_start = 398.60
Trial 23: ITI_start = 401.251
Trial 24: ITI_start = 404.895
Trial 25: ITI_start = 408.590
Trial 26: ITI_start = 411.770
Trial 27: ITI_start = 415.615
Trial 28: ITI_start = 419.80
Trial 29: ITI_start = 422.845
Trial 30: ITI_start = 435.450
Trial 31: ITI_start = 451.730
Trial 32: ITI_start = 455.715
Trial 33: ITI_start = 472.260
Trial 34: ITI_start = 483.1

,trial_number,MagEntry,On1A1,Delta,ITI_start
0,1,0.500,3.575,3.570,3.575
1,2,13.615,63.715,50.100,63.715
2,3,73.250,81.780,8.52,81.780
3,4,85.405,86.305,0.900,86.305
4,5,96.345,131.210,34.675,131.210
5,6,138.100,140.300,2.1,140.300
6,7,146.940,148.445,2.350,148.445
7,8,158.485,177.495,19.9,177.495
8,9,184.395,187.620,2.667,187.620
9,10,194.715,198.595,3.879,198.595


In [10]:
import pandas as pd

# Read the CSV files
df_data = pd.read_csv('df_659_12_12.csv')
trials_summary = pd.read_csv('trials_summary.csv')

# Function to format time as "S.MS"
def format_time(s, ms):
    return f"{s}.{ms}"

# Process each trial to find T_Premat timestamp
t_premat_values = []
for idx, row in trials_summary.iterrows():
    trial_num = int(row['trial_number'])
    start_row = int(row['start_row'])
    end_row = int(row['end_row'])
    
    # Get the slice of data for this trial (adjusting for 0-based indexing)
    trial_data = df_data.iloc[start_row-2:end_row-2]
    
    # Find Premature timestamp (Entry state)
    premature_entry = trial_data[(trial_data['Cat'] == 'Entry') & (trial_data['State'] == 'Premature')]
    if not premature_entry.empty:
        prem_s = premature_entry.iloc[0]['S']
        prem_ms = premature_entry.iloc[0]['MS']
        prem_timestamp = format_time(prem_s, prem_ms)
        t_premat_values.append(prem_timestamp)
        print(f"Trial {trial_num}: T_Premat = {prem_timestamp}")
    else:
        t_premat_values.append('0')
        print(f"Trial {trial_num}: No Premature Entry found - set to 0")

# Add T_Premat column to trials_summary
trials_summary['T_Premat'] = t_premat_values

# Save the updated CSV
trials_summary.to_csv('trials_summary.csv', index=False)
print("\nDone! Added T_Premat column to trials_summary.csv")

trials_summary[['trial_number', 'MagEntry', 'On1A1', 'ITI_start', 'T_Premat']].head(10)

Trial 1: No Premature Entry found - set to 0
Trial 2: No Premature Entry found - set to 0
Trial 3: No Premature Entry found - set to 0
Trial 4: No Premature Entry found - set to 0
Trial 5: T_Premat = 133.90
Trial 6: T_Premat = 141.930
Trial 7: No Premature Entry found - set to 0
Trial 8: T_Premat = 179.385
Trial 9: No Premature Entry found - set to 0
Trial 10: T_Premat = 200.530
Trial 11: No Premature Entry found - set to 0
Trial 12: No Premature Entry found - set to 0
Trial 13: No Premature Entry found - set to 0
Trial 14: No Premature Entry found - set to 0
Trial 15: No Premature Entry found - set to 0
Trial 16: T_Premat = 344.730
Trial 17: No Premature Entry found - set to 0
Trial 18: No Premature Entry found - set to 0
Trial 19: No Premature Entry found - set to 0
Trial 20: No Premature Entry found - set to 0
Trial 21: No Premature Entry found - set to 0
Trial 22: No Premature Entry found - set to 0
Trial 23: No Premature Entry found - set to 0
Trial 24: No Premature Entry found - 

,trial_number,MagEntry,On1A1,ITI_start,T_Premat
0,1,0.500,3.575,3.575,0
1,2,13.615,63.715,63.715,0
2,3,73.250,81.780,81.780,0
3,4,85.405,86.305,86.305,0
4,5,96.345,131.210,131.210,133.90
5,6,138.100,140.300,140.300,141.930
6,7,146.940,148.445,148.445,0
7,8,158.485,177.495,177.495,179.385
8,9,184.395,187.620,187.620,0
9,10,194.715,198.595,198.595,200.530


In [15]:
import pandas as pd

# Read the trials_summary CSV
trials_summary = pd.read_csv('trials_summary.csv')

# Function to convert "S.MS" format to total seconds as float
def time_to_seconds(time_str):
    if pd.isna(time_str) or str(time_str) == '0':
        return None
    parts = str(time_str).split('.')
    if len(parts) == 2:
        seconds = float(parts[0])
        milliseconds = float(parts[1])
        return seconds + (milliseconds / 1000.0)
    return float(time_str)

# Function to convert seconds back to "S.MS" format
def seconds_to_time(seconds):
    if seconds is None or pd.isna(seconds):
        return None
    s = int(seconds)
    ms = int((seconds - s) * 1000)
    return f"{s}.{ms}"

# Calculate ITI_premat_delta for each row
iti_premat_delta_values = []
for idx, row in trials_summary.iterrows():
    t_premat = row['T_Premat']
    
    # Check if T_Premat is '0' or 0 or invalid (handle both string and numeric)
    if pd.isna(t_premat) or str(t_premat) == '0' or t_premat == 0:
        iti_premat_delta_values.append('NaN')
        print(f"Trial {row['trial_number']}: No premature response - ITI_premat_delta = NaN")
    else:
        # Convert timestamps to seconds
        t_premat_seconds = time_to_seconds(t_premat)
        iti_start_seconds = time_to_seconds(row['ITI_start'])
        
        if t_premat_seconds is not None and iti_start_seconds is not None:
            # Calculate difference: T_Premat - ITI_start
            delta_seconds = t_premat_seconds - iti_start_seconds
            delta_formatted = seconds_to_time(delta_seconds)
            iti_premat_delta_values.append(delta_formatted)
            print(f"Trial {row['trial_number']}: T_Premat({t_premat_seconds:.3f}s) - ITI_start({iti_start_seconds:.3f}s) = {delta_formatted}")
        else:
            iti_premat_delta_values.append('NaN')
            print(f"Trial {row['trial_number']}: Missing timestamp data - ITI_premat_delta = NaN")

# Add ITI_premat_delta column to trials_summary
trials_summary['ITI_premat_delta'] = iti_premat_delta_values

# Try to save, if file is locked, show warning but continue
try:
    trials_summary.to_csv('trials_summary.csv', index=False)
    print("\nDone! Added ITI_premat_delta column to trials_summary.csv")
except PermissionError:
    print("\nWarning: Could not save to trials_summary.csv (file may be open). Close the file and run this cell again to save.")

trials_summary[['trial_number', 'ITI_start', 'T_Premat', 'ITI_premat_delta']].head(10)

Trial 1: No premature response - ITI_premat_delta = NaN
Trial 2: No premature response - ITI_premat_delta = NaN
Trial 3: No premature response - ITI_premat_delta = NaN
Trial 4: No premature response - ITI_premat_delta = NaN
Trial 5: T_Premat(133.009s) - ITI_start(131.021s) = 1.987
Trial 6: T_Premat(141.093s) - ITI_start(140.003s) = 1.90
Trial 7: No premature response - ITI_premat_delta = NaN
Trial 8: T_Premat(179.385s) - ITI_start(177.495s) = 1.889
Trial 9: No premature response - ITI_premat_delta = NaN
Trial 10: T_Premat(200.053s) - ITI_start(198.595s) = 1.457
Trial 11: No premature response - ITI_premat_delta = NaN
Trial 12: No premature response - ITI_premat_delta = NaN
Trial 13: No premature response - ITI_premat_delta = NaN
Trial 14: No premature response - ITI_premat_delta = NaN
Trial 15: No premature response - ITI_premat_delta = NaN
Trial 16: T_Premat(344.073s) - ITI_start(342.097s) = 1.975
Trial 17: No premature response - ITI_premat_delta = NaN
Trial 18: No premature response

,trial_number,ITI_start,T_Premat,ITI_premat_delta
0,1,3.575,0.000,NaN
1,2,63.715,0.000,NaN
2,3,81.780,0.000,NaN
3,4,86.305,0.000,NaN
4,5,131.210,133.900,1.987
5,6,140.300,141.930,1.90
6,7,148.445,0.000,NaN
7,8,177.495,179.385,1.889
8,9,187.620,0.000,NaN
9,10,198.595,200.530,1.457
